Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

## TEST PROBLEMS

In [40]:
import numpy as np

In [41]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [42]:
def compute_loads(solution, WEIGHTS, NUM_KNAPSACKS):
    loads = np.zeros((NUM_KNAPSACKS, WEIGHTS.shape[1]), dtype=np.int64)
    for i, k in enumerate(solution):
        if k >= 0:
            loads[k] += WEIGHTS[i]
    return loads

def is_valid(solution, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS):
    loads = compute_loads(solution, WEIGHTS, NUM_KNAPSACKS)
    return bool(np.all(loads <= CONSTRAINTS[np.newaxis, :])), loads

def objective(solution, VALUES):
    return int(VALUES[solution >= 0].sum())

def greedy_construct(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    score = VALUES / (1 + WEIGHTS.sum(axis=1))
    order = np.argsort(-score)  
    sol = np.full(len(VALUES), -1, dtype=int)
    loads = np.zeros((NUM_KNAPSACKS, WEIGHTS.shape[1]), dtype=np.int64)

    for i in order:
        w = WEIGHTS[i]
        remaining = CONSTRAINTS[np.newaxis, :] - loads
        candidates = []
        for k in range(NUM_KNAPSACKS):
            if np.all(loads[k] + w <= CONSTRAINTS):
                candidates.append((int(np.min(remaining[k] - w)), k))
        candidates.sort(reverse=True)
        for _, k in candidates:
            sol[i] = k
            loads[k] += w
            break
    return sol

In [43]:
def tabu_search(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
                iters=600, tenure=7, seed=42, max_neighbors=None):
    rng = np.random.default_rng(seed)
    N = len(VALUES)

    cur = greedy_construct(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS, rng=rng)
    cur_val = objective(cur, VALUES)
    valid, loads = is_valid(cur, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)
    if not valid:
        cur = np.full(N, -1, dtype=int)
        loads = np.zeros((NUM_KNAPSACKS, WEIGHTS.shape[1]), dtype=np.int64)
        cur_val = 0

    best = cur.copy()
    best_val = cur_val
    best_loads = loads.copy()

    tabu = {}  

    for it in range(1, iters + 1):
        for key in list(tabu.keys()):
            if tabu[key] <= it:
                del tabu[key]
        candidates = []
        items_iter = np.arange(N)
        if max_neighbors is not None and max_neighbors < N:
            items_iter = rng.choice(N, size=max_neighbors, replace=False)

        for i in items_iter:
            cur_k = cur[i]
            w = WEIGHTS[i]
            for dest in range(-1, NUM_KNAPSACKS):
                if dest == cur_k:
                    continue
                if cur_k >= 0:
                    tmp_loads = loads.copy()
                    tmp_loads[cur_k] -= w
                else:
                    tmp_loads = loads.copy()

                feasible = True
                if dest >= 0:
                    if np.any(tmp_loads[dest] + w > CONSTRAINTS):
                        feasible = False
                    else:
                        tmp_loads[dest] += w
                if not feasible:
                    continue
                new_val = cur_val
                if cur_k >= 0: new_val -= int(VALUES[i])
                if dest >= 0:  new_val += int(VALUES[i])

                move_key = (int(i), int(dest))
                is_tabu = move_key in tabu
                if (not is_tabu) or (new_val > best_val):
                    candidates.append((new_val, i, dest, tmp_loads))

        if not candidates:
            continue

        candidates.sort(key=lambda x: x[0], reverse=True)
        new_val, i_star, dest_star, loads_star = candidates[0]

        cur[i_star] = dest_star
        loads = loads_star
        cur_val = new_val

        tabu[(int(i_star), int(dest_star))] = it + tenure + rng.integers(0, 3)

        if cur_val > best_val:
            best = cur.copy()
            best_val = cur_val
            best_loads = loads.copy()

    return best, best_val, best_loads

In [44]:
solution, total_value, loads = tabu_search(
    VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
    iters=600, tenure=7, seed=42, max_neighbors=None
)

ok, _ = is_valid(solution, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)

print("VALID:", ok)
print("TOTAL VALUE:", total_value)
print("CONSTRAINTS (per knapsack):", CONSTRAINTS.tolist())
print("Per-knapsack loads:\n", loads)
print("SOLUTION (-1 means not taken):\n", solution.tolist())

for k in range(NUM_KNAPSACKS):
    idx = np.where(solution == k)[0]
    print(f"\nKnapsack {k} -> items {idx.tolist()}")
    print("  value:", int(VALUES[idx].sum()))
    print("  load :", loads[k].tolist())

VALID: True
TOTAL VALUE: 1065
CONSTRAINTS (per knapsack): [614, 496]
Per-knapsack loads:
 [[329 365]
 [339 367]
 [317 340]]
SOLUTION (-1 means not taken):
 [2, 1, 0, 1, 2, 2, 2, 2, 1, 0, 2, 0, 2, 0, 1, 1, 0, 2, 1, 0]

Knapsack 0 -> items [2, 9, 11, 13, 16, 19]
  value: 343
  load : [329, 365]

Knapsack 1 -> items [1, 3, 8, 14, 15, 18]
  value: 372
  load : [339, 367]

Knapsack 2 -> items [0, 4, 5, 6, 7, 10, 12, 17]
  value: 350
  load : [317, 340]


In [45]:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [46]:
solution, total_value, loads = tabu_search(
    VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
    iters=600, tenure=7, seed=42, max_neighbors=None
)

ok, _ = is_valid(solution, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)

print("VALID:", ok)
print("TOTAL VALUE:", total_value)
print("CONSTRAINTS (per knapsack):", CONSTRAINTS.tolist())
print("Per-knapsack loads:\n", loads)
print("SOLUTION (-1 means not taken):\n", solution.tolist())

for k in range(NUM_KNAPSACKS):
    idx = np.where(solution == k)[0]
    print(f"\nKnapsack {k} -> items {idx.tolist()}")
    print("  value:", int(VALUES[idx].sum()))
    print("  load :", loads[k].tolist())

VALID: True
TOTAL VALUE: 39270
CONSTRAINTS (per knapsack): [5837, 5754, 9351, 3205, 4450, 3447, 3952, 9256, 9525, 2357]
Per-knapsack loads:
 [[3157 3829 4286 3100 3312 2997 2131 3681 4152 2310]
 [4002 3279 4010 2978 1679 1848 2809 1206 2416 2214]
 [3271 3830 2981 3007 2573 3443 3919 3338 2807 2154]
 [2711 3578 2389 3197 3766 3114 3704 2554 1715 2296]
 [3430 2301 3132 3069 3815 3397 1966 1245 2992 2325]
 [3666 2357 2131 3178 2922 2750 2500 2115 3068 2246]
 [2853 2516 4581 2983 4389 2751 2684 3353 3925 2283]
 [1858 4554 1419 2979 4134 3172 3070 3087 3361 2269]
 [3104 3753 2181 2868 4132 3438 3781 2379 3850 2245]
 [3293 2981 3535 3051 2733 3324 2738 1904 2722 2180]]
SOLUTION (-1 means not taken):
 [-1, 4, -1, 8, 4, 0, 1, 0, 7, -1, -1, 4, 8, 8, 3, 7, -1, -1, 6, 4, 7, -1, -1, 3, 6, -1, -1, 3, 0, 4, 5, -1, -1, 2, 5, 9, 9, 8, -1, 6, -1, 1, 4, 8, -1, 6, -1, 2, 7, -1, -1, 3, 0, -1, -1, -1, 7, -1, -1, 1, 8, 5, -1, 1, 9, -1, 0, -1, 0, -1, 2, -1, 7, -1, 1, -1, -1, -1, 7, 9, 9, -1, -1, 2, 9, 8, 6, 

In [47]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [48]:
def compute_loads(solution, WEIGHTS, K):
    N, D = WEIGHTS.shape
    loads = np.zeros((K, D), dtype=np.int64)
    mask = solution >= 0
    if np.any(mask):
        for i, k in zip(np.nonzero(mask)[0], solution[mask]):
            loads[k] += WEIGHTS[i]
    return loads

def is_valid(solution, WEIGHTS, CONSTRAINTS, K):
    loads = compute_loads(solution, WEIGHTS, K)
    return bool(np.all(loads <= CONSTRAINTS[np.newaxis, :])), loads

def greedy_seed(VALUES, WEIGHTS, CONSTRAINTS, K, seed=42):
    rng = np.random.default_rng(seed)
    N, D = WEIGHTS.shape
    sol = np.full(N, -1, dtype=int)
    loads = np.zeros((K, D), dtype=np.int64)
    slack = CONSTRAINTS[np.newaxis, :].astype(np.int64) - loads
    order = np.argsort(-(VALUES / (1 + WEIGHTS.sum(axis=1))))
    for i in order:
        w = WEIGHTS[i]
        fits = np.all(w <= slack, axis=1)
        dests = np.flatnonzero(fits)
        if dests.size:
            k = int(dests[rng.integers(0, len(dests))])
            sol[i] = k
            loads[k] += w
            slack[k] -= w
    return sol, loads

In [49]:
def tabu_search_fast(VALUES, WEIGHTS, CONSTRAINTS, K, iters=300, tenure=7, seed=42, max_neighbors=1200, p=3, stall_limit=200):
    rng = np.random.default_rng(seed)
    N, D = WEIGHTS.shape
    cur, loads = greedy_seed(VALUES, WEIGHTS, CONSTRAINTS, K, seed=seed)
    slack = CONSTRAINTS[np.newaxis, :].astype(np.int64) - loads
    cur_val = int(VALUES[cur >= 0].sum())
    best = cur.copy()
    best_val = cur_val
    best_loads = loads.copy()
    tabu = {}
    stall = 0

    for it in range(1, iters + 1):
        for key in list(tabu.keys()):
            if tabu[key] <= it:
                del tabu[key]

        items_iter = rng.choice(N, size=min(max_neighbors, N), replace=False)
        improved = False

        for i in items_iter:
            cur_k = cur[i]
            w_i = WEIGHTS[i]
            if cur_k >= 0:
                slack[cur_k] += w_i

            fits_mask = np.all(w_i <= slack, axis=1)
            if cur_k >= 0:
                fits_mask[cur_k] = False
            dests = np.flatnonzero(fits_mask)

            if dests.size:
                after = slack[dests] - w_i
                comfort = np.min(after, axis=1)
                p_eff = min(p, comfort.size)
                cand_dests = dests[np.argpartition(-comfort, kth=p_eff-1)[:p_eff]]
            else:
                cand_dests = np.empty(0, dtype=int)

            if cur_k >= 0:
                slack[cur_k] -= w_i

            try_dests = [-1]
            if cand_dests.size:
                try_dests += cand_dests.tolist()

            moved = False
            for dest in try_dests:
                if dest == cur_k:
                    continue
                new_val = cur_val - (VALUES[i] if cur_k >= 0 else 0) + (VALUES[i] if dest >= 0 else 0)
                if (i, dest) in tabu and new_val <= best_val:
                    continue
                if dest >= 0 and np.any(slack[dest] - w_i < 0):
                    continue

                if cur_k >= 0:
                    loads[cur_k] -= w_i
                    slack[cur_k] += w_i
                if dest >= 0:
                    loads[dest] += w_i
                    slack[dest] -= w_i

                cur[i] = dest
                cur_val = new_val
                tabu[(i, dest)] = it + tenure + rng.integers(0, 3)

                if cur_val > best_val:
                    best = cur.copy()
                    best_val = cur_val
                    best_loads = loads.copy()

                improved = True
                moved = True
                break

            if moved:
                break

        if improved:
            stall = 0
        else:
            stall += 1
            if stall >= stall_limit:
                break

    return best, best_val, best_loads

In [50]:
solution, total_value, loads = tabu_search_fast(
    VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS,
    iters=300, tenure=7, seed=42, max_neighbors=1200, p=3, stall_limit=200
)

ok, loads_check = is_valid(solution, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)

print("VALID:", ok)
print("TOTAL VALUE:", total_value)
print("CONSTRAINTS (per knapsack):", CONSTRAINTS.tolist())
print("Per-knapsack loads:\n", loads_check)
print("SOLUTION (-1 means not taken):\n", solution.tolist())

for k in range(NUM_KNAPSACKS):
    idx = np.where(solution == k)[0]
    print(f"\nKnapsack {k} -> items {idx.tolist()}")
    print("  value:", int(VALUES[idx].sum()) if idx.size else 0)
    print("  load :", loads_check[k].tolist())


VALID: True
TOTAL VALUE: 1790578
CONSTRAINTS (per knapsack): [79834, 52005, 17527, 30007, 59295, 98921, 12881, 36174, 63958, 90566, 25740, 12251, 27746, 78136, 90704, 39249, 23072, 69010, 17326, 29277, 42886, 48016, 74931, 93479, 30898, 18137, 75557, 56030, 75506, 90246, 42093, 34668, 22075, 79709, 98971, 15032, 57157, 21369, 76865, 11589, 82666, 78074, 68715, 85558, 61242, 80943, 26905, 84378, 85069, 77221, 16434, 28156, 17827, 83548, 72921, 27512, 41395, 23936, 72519, 96465, 12904, 72709, 81023, 81015, 28959, 98041, 90482, 89802, 76408, 29899, 51171, 94767, 78351, 57860, 58504, 40545, 97025, 99185, 42888, 54965, 24945, 51281, 31276, 61913, 96714, 93034, 72901, 95222, 37857, 53461, 15463, 71355, 64979, 70086, 48286, 38026, 60109, 78500, 56643, 69915]
Per-knapsack loads:
 [[10347 12458 14193 ...  9821 11989 12963]
 [ 9420 13181 13380 ... 12620 11799 12278]
 [10797 10429  9690 ... 11436 11504 11326]
 ...
 [11674 11995 12911 ... 14993 11375 15609]
 [12458 11802 11603 ...  9883 11397 1035